Importing Data


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

Random stuff

In [ ]:
data_dir = 'Data'
data_arr_all = []
for i in os.listdir(os.path.join(data_dir)):
    for ii in os.listdir(os.path.join(data_dir, i)):
        print(os.path.join(data_dir, i, ii))
        data_arr_all.append(os.path.join(data_dir, i, ii))

In [ ]:
# Shuffle Data
np.random.shuffle(data_arr_all)

In [ ]:
print(data_arr_all)

In [ ]:
# Divide data into train, test and cross validation
train_data = data_arr_all[:int(len(data_arr_all)*0.7)]
cross_validation_data = data_arr_all[int(len(data_arr_all)*0.7):int(len(data_arr_all)*0.85)]
test_data = data_arr_all[int(len(data_arr_all)*0.85):]
print(train_data)
print(cross_validation_data)
print(test_data)


In [ ]:
for i in train_data:
    print(i, end=' ')
    print(cv2.imread(i).shape)
    plt.imshow(cv2.imread(i))
    plt.show()


Loading Data


In [ ]:
tf.data.Dataset??